# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
gmaps.configure(api_key=g_key)

In [3]:

df = pd.read_csv("output_data/cities.csv")
df.head()

,city_id,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
0,0,monte alegre,-1.998911,-54.072769,Clouds,74.34,93,1009,1.05,95,BR
1,1,san cristobal,7.775666,-72.221415,Clouds,75.07,95,1014,3.06,69,VE
2,2,port elizabeth,-33.961705,25.620752,Clear,68.11,88,1015,11.50,0,ZA
3,3,ostrovnoy,68.054445,39.512938,Clouds,24.48,93,988,13.02,100,RU
4,4,mount gambier,-37.824670,140.782007,Clouds,71.22,55,1008,8.10,16,AU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# get data
data = df.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)



# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

In [5]:
city_df = df.loc[(df["temperature"] < 80) & (df["temperature"] > 70) \
                                    & (df["wind_speed"] < 10) \
                                    & (df["cloudiness"] == 0)].dropna()
city_df

,city_id,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
38,38,hobart,-42.882509,147.328123,Clear,72.39,72,1000,5.75,0,AU
107,107,saldanha,-32.997778,17.945556,Clear,70.07,82,1007,9.86,0,ZA
169,169,atar,20.518194,-13.054393,Clear,72.59,20,1018,8.57,0,MR
360,360,palmetto,27.521427,-82.572319,Clear,72.97,65,1019,3.44,0,US
382,382,narrabri,-30.329615,149.781449,Clear,79.45,56,1011,1.01,0,AU
470,470,axim,4.865362,-2.242169,Clear,78.66,70,1012,5.23,0,GH


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
hotel_df = city_df[["city","latitude", "longitude"]].copy()
hotel_df["hotel"] = ""
hotel_df

,city,latitude,longitude,hotel
38,hobart,-42.882509,147.328123,
107,saldanha,-32.997778,17.945556,
169,atar,20.518194,-13.054393,
360,palmetto,27.521427,-82.572319,
382,narrabri,-30.329615,149.781449,
470,axim,4.865362,-2.242169,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df
    

,city,latitude,longitude,hotel
38,hobart,-42.882509,147.328123,St Ives Hobart Accommodation Tasmania
107,saldanha,-32.997778,17.945556,Blue Bay Lodge
169,atar,20.518194,-13.054393,Oumou Elghoura
360,palmetto,27.521427,-82.572319,Courtyard by Marriott Bradenton Sarasota/River...
382,narrabri,-30.329615,149.781449,Crossroads Hotel
470,axim,4.865362,-2.242169,Axim Beach Resort And Spa


In [25]:
hotel_all =pd.merge(hotel_df, city_df,)
hotel_all


,city,latitude,longitude,hotel,city_id,weather,temperature,humidity,pressure,wind_speed,cloudiness,country
0,hobart,-42.882509,147.328123,St Ives Hobart Accommodation Tasmania,38,Clear,72.39,72,1000,5.75,0,AU
1,saldanha,-32.997778,17.945556,Blue Bay Lodge,107,Clear,70.07,82,1007,9.86,0,ZA
2,atar,20.518194,-13.054393,Oumou Elghoura,169,Clear,72.59,20,1018,8.57,0,MR
3,palmetto,27.521427,-82.572319,Courtyard by Marriott Bradenton Sarasota/River...,360,Clear,72.97,65,1019,3.44,0,US
4,narrabri,-30.329615,149.781449,Crossroads Hotel,382,Clear,79.45,56,1011,1.01,0,AU
5,axim,4.865362,-2.242169,Axim Beach Resort And Spa,470,Clear,78.66,70,1012,5.23,0,GH


In [26]:
infos = []

for indx, row in hotel_all.iterrows():
    info_box_template = f"""<dl>
                        <dt>Country:</dt><dd>{row.country}</dd>
                        <dt>City:</dt><dd>{row.city}</dd>    
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                         <dt>Hotel Name:</dt><dd>{row.hotel}</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Country:</dt><dd>AU</dd>\n                        <dt>City:</dt><dd>hobart</dd>    \n                        <dt>Location:</dt><dd>(-42.8825088, 147.3281233)</dd>\n                         <dt>Hotel Name:</dt><dd>St Ives Hobart Accommodation Tasmania</dd>\n                        </dl>\n                        '

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [27]:
 # get data
data = df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_all.loc[:, ["latitude", "longitude"]]
# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(data, info_box_content=info_box_template)
fig.add_layer(markers)

# show the plot
fig 

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…